# Training CTKG Using TransE_L2
Adapted from the notebook of DRKG: https://github.com/gnn4dr/DRKG
This notebook shows how to train CTKG embeddings using TransE_L2

## Install DGL-KE
Before training the model, we need to install dgl and dgl-ke packages as well as other dependencies. 

In [1]:
import GPUtil
GPUtil.getAvailable()

[0]

In [2]:
!pip3 install dglke

Defaulting to user installation because normal site-packages is not writeable


## Prepare train/valid/test set
Before training, we need to split the original ctkg into train/valid/test set with a 9:0.5:0.5 manner.

In [3]:
import pandas as pd
import numpy as np
import sys
sys.path.insert(1, '../utils')
ctkg_file = '../rawdata/ctkg.tsv'

df = pd.read_csv(ctkg_file, sep="\t")
triples = df.values.tolist()

In [4]:
num_triples = len(triples)
num_triples

7329237

In [5]:
# Please make sure the output directory exist.
seed = np.arange(num_triples)
np.random.shuffle(seed)

train_cnt = int(num_triples * 0.9)
valid_cnt = int(num_triples * 0.05)
train_set = seed[:train_cnt]
train_set = train_set.tolist()
valid_set = seed[train_cnt:train_cnt+valid_cnt].tolist()
test_set = seed[train_cnt+valid_cnt:].tolist()

with open("./train/ctkg_train.tsv", 'w+') as f:
    for idx in train_set:
        f.writelines("{}\t{}\t{}\n".format(triples[idx][0], triples[idx][1], triples[idx][2]))
        
with open("./train/ctkg_valid.tsv", 'w+') as f:
    for idx in valid_set:
        f.writelines("{}\t{}\t{}\n".format(triples[idx][0], triples[idx][1], triples[idx][2]))

with open("./train/ctkg_test.tsv", 'w+') as f:
    for idx in test_set:
        f.writelines("{}\t{}\t{}\n".format(triples[idx][0], triples[idx][1], triples[idx][2]))

## Training TransE_l2 model
We can training the TransE_l2 model by simplying using DGL-KE command line. For more information about using DGL-KE please refer to https://github.com/awslabs/dgl-ke.

In [ ]:
!DGLBACKEND=pytorch dglke_train --dataset CTKG --data_path ./train --data_files ctkg_train.tsv ctkg_valid.tsv ctkg_test.tsv --format 'raw_udd_hrt' --model_name TransE_l2 --batch_size 2048 \
--neg_sample_size 256 --hidden_dim 200 --gamma 12.0 --lr 0.1 --max_step 100000 --log_interval 1000 --batch_size_eval 16 -adv --regularization_coef 1.00E-07 --test --num_thread 1 --gpu 0 --num_proc 1 --neg_sample_size_eval 10000

## Get Entity and Relation Embeddings
The resulting model, i.e., the entity and relation embeddings can be found under ./ckpts. (Please refer to the first line of the training log for the specific location.)

The overall process will generate 4 important files:

  - Entity embedding: ./ckpts/<model\_name>_<dataset\_name>_<run_\id>/xxx\_entity.npy
  - Relation embedding: ./ckpts/<model\_name>_<dataset\_name>_<run\_id>/xxx\_relation.npy
  - The entity id mapping, formated in <entity\_name> <entity\_id> pair: <data\_path>/entities.tsv
  - The relation id mapping, formated in <relation\_name> <relation\_id> pair: <data\_path>/relations.tsv

In [ ]:
!ls ./ckpts/TransE_l2_CTKG_0/
!ls ./train/

## A Glance of the Entity and Relation Embeddings

In [ ]:
node_emb = np.load('./ckpts/TransE_l2_CTKG_0/CTKG_TransE_l2_entity.npy')
relation_emb = np.load('./ckpts/TransE_l2_CTKG_0/CTKG_TransE_l2_relation.npy')

print(node_emb.shape)
print(relation_emb.shape)